# Preprocessing and Trining Model

In [1]:
import tensorflow as tf
import os

In [2]:
!pip install -q kaggle

In [3]:
# Upload Kaggle API
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"najibullohasror","key":"7b1c0480d911c3aa453e581eff98e4c6"}'}

In [4]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [5]:
! kaggle datasets download -d najibullohasror/hitani-dataset

100% 7.53G/7.56G [01:05<00:00, 125MB/s]
100% 7.56G/7.56G [01:05<00:00, 124MB/s]


In [6]:
import os 
import zipfile

local_zip = '/content/hitani-dataset.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content/')
zip_ref.close()

In [7]:
import numpy as np
import pickle
import cv2
from os import listdir
from tensorflow import keras
from sklearn.preprocessing import LabelBinarizer
from keras.models import Sequential
from keras.layers import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation, Flatten, Dropout, Dense
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from keras.preprocessing import image
from keras.preprocessing.image import img_to_array
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [8]:
EPOCHS = 50
INIT_LR = 1e-3
BS = 32
default_image_size = tuple((100,100))
image_size = 0
directory_root = '/content/CornLeafDiseases'
width=100
height=100
depth=3

In [9]:
# Function to convert images to array
def convert_image_to_array(image_dir):
    try:
        image = cv2.imread(image_dir)
        if image is not None :
            image = cv2.resize(image, default_image_size)   
            return img_to_array(image)
        else :
            return np.array([])
    except Exception as e:
        print(f"Error : {e}")
        return None

In [10]:
# Fetch images from directory
image_list, label_list = [], []
try:
    print("[INFO] Loading images ...")
    root_dir = listdir(directory_root)
    for directory in root_dir :
        # remove .Hi_Tani from list
        if directory == ".Hi_Tani" :
            root_dir.remove(directory)

    for plant_folder in root_dir :
        plant_disease_folder_list = listdir(f"{directory_root}/{plant_folder}")
        
        for disease_folder in plant_disease_folder_list :
            # remove .Hi_Tani from list
            if disease_folder == ".Hi_Tani" :
                plant_disease_folder_list.remove(disease_folder)

        for plant_disease_folder in plant_disease_folder_list:
            print(f"[INFO] Processing {plant_disease_folder} ...")
            plant_disease_image_list = listdir(f"{directory_root}/{plant_folder}/{plant_disease_folder}/")
                
            for single_plant_disease_image in plant_disease_image_list :
                if single_plant_disease_image == ".Hi_Tani" :
                    plant_disease_image_list.remove(single_plant_disease_image)

            for image in plant_disease_image_list[:250]:
                image_directory = f"{directory_root}/{plant_folder}/{plant_disease_folder}/{image}"
                if image_directory.endswith(".jpg") == True or image_directory.endswith(".JPG") == True:
                    image_list.append(convert_image_to_array(image_directory))
                    label_list.append(plant_disease_folder)
    print("[INFO] Image loading completed")  
except Exception as e: 
    print(f"Error : {e}")

[INFO] Loading images ...
[INFO] Processing Corn_(maize)___Northern_Leaf_Blight ...
[INFO] Processing Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot ...
[INFO] Processing Corn_(maize)___Common_rust_ ...
[INFO] Processing Corn_(maize)___healthy ...
[INFO] Image loading completed


In [13]:
# Get Size of Processed Image
image_size = len(image_list)

In [14]:
# Transform Image Labels uisng Scikit Learn's LabelBinarizer
label_binarizer = LabelBinarizer()
image_labels = label_binarizer.fit_transform(label_list)
pickle.dump(label_binarizer,open('label_transform.pkl', 'wb'))
n_classes = len(label_binarizer.classes_)

In [15]:
# Print the classes
print(label_binarizer.classes_)

['Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot'
 'Corn_(maize)___Common_rust_' 'Corn_(maize)___Northern_Leaf_Blight'
 'Corn_(maize)___healthy']


In [16]:
np_image_list = np.array(image_list, dtype=np.float16) / 225.0

In [17]:
print("[INFO] Spliting data to train, test")
x_train, x_test, y_train, y_test = train_test_split(np_image_list, image_labels, test_size=0.2, random_state = 42) 

[INFO] Spliting data to train, test


In [18]:
aug = ImageDataGenerator(
    rotation_range=25, width_shift_range=0.1,
    height_shift_range=0.1, shear_range=0.2, 
    zoom_range=0.2,horizontal_flip=True, 
    fill_mode="nearest")

In [19]:
# Build Model Stucture 
model = Sequential()
inputShape = (height, width, depth)
chanDim = -1
if K.image_data_format() == "channels_first":
    inputShape = (depth, height, width)
    chanDim = 1
model.add(Conv2D(32, (3, 3), padding="same",input_shape=inputShape))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(128, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(Conv2D(128, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(1024))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(n_classes))
model.add(Activation("softmax"))

# Model Summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 100, 100, 32)      896       
                                                                 
 activation (Activation)     (None, 100, 100, 32)      0         
                                                                 
 batch_normalization (BatchN  (None, 100, 100, 32)     128       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 33, 33, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 33, 33, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 33, 33, 64)        1

In [20]:
#Compiling Model
opt = Adam(learning_rate=INIT_LR, decay=INIT_LR / EPOCHS)
# distribution
model.compile(loss="binary_crossentropy", optimizer=opt,metrics=["accuracy"])
# train the network
print("[INFO] training network...")

[INFO] training network...


In [21]:
#Fitting Model
history = model.fit_generator(
    aug.flow(x_train, y_train, batch_size=BS),
    validation_data=(x_test, y_test),
    steps_per_epoch=len(x_train) // BS,
    epochs=EPOCHS, verbose=1
    )

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


Epoch 1/50
25/25 [==============================] - 15s 100ms/step - loss: 0.5733 - accuracy: 0.7225 - val_loss: 0.6348 - val_accuracy: 0.3800
Epoch 2/50
25/25 [==============================] - 2s 79ms/step - loss: 0.2961 - accuracy: 0.8537 - val_loss: 0.7846 - val_accuracy: 0.3750
Epoch 3/50
25/25 [==============================] - 2s 80ms/step - loss: 0.2044 - accuracy: 0.8863 - val_loss: 1.0592 - val_accuracy: 0.2200
Epoch 4/50
25/25 [==============================] - 2s 81ms/step - loss: 0.1646 - accuracy: 0.9087 - val_loss: 0.8805 - val_accuracy: 0.3350
Epoch 5/50
25/25 [==============================] - 2s 81ms/step - loss: 0.1274 - accuracy: 0.9212 - val_loss: 1.0191 - val_accuracy: 0.2200
Epoch 6/50
25/25 [==============================] - 2s 81ms/step - loss: 0.1209 - accuracy: 0.9162 - val_loss: 0.8309 - val_accuracy: 0.2600
Epoch 7/50
25/25 [==============================] - 2s 82ms/step - loss: 0.1301 - accuracy: 0.9225 - val_loss: 0.8941 - val_accuracy: 0.3450
Epoch 8/50


In [22]:
# Model Accuracy
print("[INFO] Calculating model accuracy")
scores = model.evaluate(x_test, y_test)
print(f"Test Accuracy: {scores[1]*100}")

[INFO] Calculating model accuracy
7/7 [==============================] - 0s 8ms/step - loss: 0.0845 - accuracy: 0.9500
Test Accuracy: 94.9999988079071


In [24]:
# Save the model using TensorFlow SavedModel format
model.save('corn_disease_model.h5')

In [25]:
model2 = tf.keras.models.load_model('/content/corn_disease_model.h5', compile=False)

In [26]:
# Conver Model to TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model2)
tflite_model = converter.convert()

with open('corn_model.tflite', 'wb') as f:
  f.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmpko_2jsef/assets


In [27]:
scores = model.evaluate(x_train, y_train)
print(f"Test Accuracy: {scores[1]*100}")

25/25 [==============================] - 0s 8ms/step - loss: 0.0256 - accuracy: 0.9787
Test Accuracy: 97.87499904632568


# Predict 

In [31]:
import keras
import cv2
import matplotlib.pyplot as plt

In [32]:
import numpy as np
CATEGORIES = ['Gray_Leaf_Spot', 'Common_Rust', 'Northern_Leaf_Blight', 'Healthy']

def image(path):
    img = cv2.imread(path)
    new_arr = cv2.resize(img,(100, 100))
    new_arr = np.array(new_arr/255)
    new_arr = new_arr.reshape(-1, 100, 100, 3)
    return new_arr

model = keras.models.load_model('/content/corn_disease_model.h5')

In [33]:
prediction = model.predict(image('/content/Northern Leaf Blight (1).jpg'))

print(CATEGORIES[prediction.argmax()])

Northern_Leaf_Blight
